In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(api_key=groq_api_key, model="gemma2-9b-It")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x107c9ea50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x107c9f620>, model_name='gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hello, my name is Prathmesh")])


AIMessage(content="Hello Prathmesh, it's nice to meet you! 👋\n\nWhat can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 17, 'total_tokens': 45, 'completion_time': 0.050909091, 'prompt_time': 0.00124367, 'queue_time': 0.256687289, 'total_time': 0.052152761}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--2a08bc01-3c2e-4c5f-9405-0028666ed07b-0', usage_metadata={'input_tokens': 17, 'output_tokens': 28, 'total_tokens': 45})

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
    HumanMessage(content="Hello, my name is Prathmesh"),
    AIMessage(content="Hello Prathmesh, it's nice to meet you! 👋\n\nWhat can I do for you today? 😊 \n"),
    HumanMessage(content="What's my Name?")
    ]
)

AIMessage(content='Your name is Prathmesh, as you told me earlier. 😊  \n\nIs there anything else I can help you with?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 59, 'total_tokens': 89, 'completion_time': 0.054545455, 'prompt_time': 0.00218781, 'queue_time': 0.247859409, 'total_time': 0.056733265}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f381548f-b785-4d81-a37a-ebf4634d341e-0', usage_metadata={'input_tokens': 59, 'output_tokens': 30, 'total_tokens': 89})

In [6]:
#message History


In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_msg_history = RunnableWithMessageHistory(model,get_session_history)

In [9]:
config = {'configurable':{'session_id':'chat1'}}

In [12]:
res = with_msg_history.invoke(
    [HumanMessage(content="Hello, my name is Prathmesh")],
    config=config
)

In [13]:
res.content

"Hello Prathmesh, it's nice to meet you! \n\nWhat can I help you with today? 😊  \n"

In [14]:
with_msg_history.invoke(
    [HumanMessage(content="What is my Name?")],
    config=config
)

AIMessage(content='Your name is Prathmesh.  \n\nI remember that you told me earlier!  😄  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 101, 'total_tokens': 125, 'completion_time': 0.043636364, 'prompt_time': 0.002654199, 'queue_time': 0.256965359, 'total_time': 0.046290563}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1bf5c698-e15e-4548-9dcd-06f4a5d4b09e-0', usage_metadata={'input_tokens': 101, 'output_tokens': 24, 'total_tokens': 125})

In [16]:
## trying to change the config to see if it remembers my name
config1 = {'configurable':{'session_id':'chat2'}}

In [17]:
res = with_msg_history.invoke(
    [HumanMessage(content="What is my Name?")],
    config=config1
)

In [18]:
res.content

"As an AI, I have no memory of past conversations and no access to personal information about you, including your name.\n\nIf you'd like to tell me your name, I'm happy to use it! 😊\n"

Prompt Templates

Helps to turn raw user info into a format that the LLM can work with,


In [22]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Yor are an expert AI Assistant, help user with their queries"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [23]:
chain.invoke({"messages":[HumanMessage(content="Hello, my name is Prathmesh")]})

AIMessage(content="Hello Prathmesh! It's nice to meet you. 👋\n\nI'm ready to help with your queries. What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 29, 'total_tokens': 68, 'completion_time': 0.070909091, 'prompt_time': 0.00140806, 'queue_time': 0.244668169, 'total_time': 0.072317151}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b1977228-7786-4f2b-9cfc-5b3caeb7a969-0', usage_metadata={'input_tokens': 29, 'output_tokens': 39, 'total_tokens': 68})

In [24]:
with_msg_history = RunnableWithMessageHistory(chain,get_session_history)

In [28]:
config3 = {'configurable':{'session_id':'chat3'}}

In [29]:
res = with_msg_history.invoke(
    [HumanMessage(content="Hello, my name is Prathmesh")],
    config=config3
)

In [30]:
res.content

"Hello Prathmesh, it's nice to meet you! 👋\n\nIs there anything I can help you with today?  I'm ready for any questions or tasks you have. 😄  \n"

In [31]:
## Adding more Complexity
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Yor are an expert AI Assistant, help user with their queries in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ])

In [32]:
chain = prompt | model

In [33]:
res = chain.invoke({"messages": [HumanMessage(content="Hello, my name is Prathmesh")], "language":"Hindi"})

res.content

'नमस्ते प्रथमेश! 😊  \n\nमैं आपकी सहायता करने के लिए तैयार हूँ। आप मुझसे कोई भी प्रश्न पूछ सकते हैं और मैं अपनी पूरी कोशिश करूँगा कि उसका उत्तर आपको समझ में आए। \n\nक्या मैं आपकी किसी मदद कर सकता हूँ? \n\n'

Let's Wrap this more complicated chain in message history class, This time because there are multiple keys in the 

In [34]:
with_msg_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [35]:
config = {'configurable':{'session_id':'chat4'}}
resp = with_msg_history.invoke(
    {"messages": [HumanMessage(content="Hello, my name is Prathmesh")], "language":"Hindi"},
    config=config
)

In [36]:
resp.content

'Namaste Prathmesh! 😊  \n\nमुझे बहुत खुशी है कि आप यहाँ हैं। मैं आपकी सहायता करने के लिए तैयार हूँ। आपके कोई सवाल हैं? \n\nआप मुझसे कुछ भी पूछ सकते हैं, जैसे:\n\n* जानकारी खोजने के लिए\n* लेख लिखने में मदद के लिए\n* भाषा अनुवाद के लिए\n* रचनात्मकता के लिए \n* या बस बातचीत करने के लिए \n\n\nमैं आपकी मदद करने के लिए यहाँ हूँ! \n\n'

In [37]:
resp = with_msg_history.invoke(
    {"messages": [HumanMessage(content="what is my name")], "language":"Hindi"},
    config=config
)

In [38]:
resp.content

'आपका नाम Prathmesh है। 😊 \n'

# Managing the Conversation History
If let unattended, the the list of messages will grow unbounded and potentially overflow the context window of the llm

In [39]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, trim_messages

In [46]:
trim = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [47]:
messages = [
    SystemMessage(content="you are an expert assistant"),
    HumanMessage(content="Hello, my name is Prathmesh"),
    AIMessage(content="Hello Prathmesh, it's nice to meet you!"),
    HumanMessage(content="Can you tell me what is my name?"),
    AIMessage(content="Your name is Prathmesh."),
    HumanMessage(content="What is 4+4"),
    AIMessage(content="4+4 is 8"),
    ]

In [48]:
trim.invoke(messages)

[SystemMessage(content='you are an expert assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Can you tell me what is my name?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Your name is Prathmesh.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 4+4', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4+4 is 8', additional_kwargs={}, response_metadata={})]

In [49]:
from operator import itemgetter
from langchain_core.runnables import RunnableSequence, RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trim)
    | prompt
    | model
)

In [54]:
response = chain.invoke(
    {"messages": messages + [HumanMessage(content="Can you tell me what is my name?")],
    "language":"English"}
    )

response.content

"As an AI, I don't have access to any personal information about you, including your name. Can you tell me your name? 😊  \n\n"

In [55]:
with_msg_hist = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

In [56]:
res = with_msg_hist.invoke(
    {"messages": messages + [HumanMessage(content="Can you tell me what is my name?")],
    "language":"English"},
    config={'configurable':{'session_id':'chat5'}}
)
res.content

"As an AI, I don't have access to any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"